# LSTM-RNN

In [47]:
from dotenv import load_dotenv
import sys
import warnings

from __init__ import get_base_path
import data.load_data as load_data
from model_utils import FeatureStore, NonZeroLabelEncoder, grab_bag_train_test_split, pad_collate, SignalClassificationDataset, DynamicLSTM

import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

load_dotenv()
warnings.filterwarnings('ignore')


In [48]:
rnn_data_dir = get_base_path() + os.getenv('RNN_DATA_DIR')

In [49]:
random_seed = os.getenv('RANDOM_SEED')
torch.manual_seed(random_seed)

### Load Data

In [50]:
# csv_file = load_data.combined_csv
csv_file = load_data.combined_sample_csv

if os.path.exists(csv_file):
    pass
else:
    load_data.__main__()
df = pd.read_csv(csv_file,index_col=0)

# datatyping that may not be retained by csv
dp = load_data.DataPreprocessor(df)
df = dp.cast_data_types().get_dataframe() 

### Features

In [51]:
features = FeatureStore(df)

X_features = features.R1_waves
y_features = features.y_tertiary
# X_features = ['R1_Phase_A_power_wave']

X = df.loc[:,X_features + ['sample_id']]
y = df.loc[:,y_features]

### Train Test Split

In [52]:
X_train, y_train, X_test, y_test, train_ids, test_ids = grab_bag_train_test_split(X, y, df['sample_id'], return_ids=True)
for i in X_train, y_train, X_test, y_test, train_ids, test_ids:
    print(len(i))

20248
20248
5696
5696
148
36


### MinMaxScaler

In [53]:
scl = MinMaxScaler()
scl.fit(X_train[X_features])
X_train[X_features], X_test[X_features] = scl.transform(X_train[X_features]), scl.transform(X_test[X_features])

### Encoding

* Label Encode Y Classes
* Because we will use zero padding we use a custom label encoder

In [54]:
# enc = LabelEncoder()
enc = NonZeroLabelEncoder()

enc.fit(y_train)
y_train, y_test = enc.transform(y_train), enc.transform(y_test)

print(enc.mappings)

((1, 'attack'), (2, 'natural'), (3, 'no event'))


### Dataset, DataLoader

1. extract individual signal lengths
2. pack using PyTorch padded sequences which allow variable length sequences to be sent to LSTM, without computation over the padded zeroes

In [55]:
y_class_encodings = enc.encodings
k_classes = len(y_class_encodings)
x_train_vectors, x_test_vectors, y_train_vectors, y_test_vectors = [],[],[],[]

for encoding in y_class_encodings:
    # subset train and test dataframes for the class label (ie '0'/'attack')
    train_filt, test_filt = y_train == encoding, y_test == encoding
    X_train_filt, X_test_filt = X_train.loc[train_filt], X_test.loc[test_filt]
    train_samples, test_samples = X_train_filt['sample_id'].unique(), X_test_filt['sample_id'].unique()

    # partition result by sample_id, and accumulate vectors
    for sample in train_samples:
        s = X_train_filt.loc[X_train_filt['sample_id'] == sample,X_features]
        x_train_vectors += [torch.tensor(s.to_numpy(),dtype=torch.float)]
        # y_train_vectors += [torch.tensor(np.full(shape=(len(s),),fill_value=encoding),dtype=torch.float)]
        y_train_vectors += [torch.tensor(np.array([encoding]),dtype=torch.float)]

    for sample in test_samples:
        s = X_test_filt.loc[X_test_filt['sample_id'] == sample,X_features]
        x_test_vectors += [torch.tensor(s.to_numpy(),dtype=torch.float)]
        # y_test_vectors += [torch.tensor(np.full(shape=(len(s),),fill_value=encoding),dtype=torch.float)]
        y_test_vectors += [torch.tensor(np.array([encoding]),dtype=torch.float)]


In [56]:
# dataset
train_dataset = SignalClassificationDataset(signals=x_train_vectors,labels=y_train_vectors)
test_dataset = SignalClassificationDataset(signals=x_test_vectors,labels=y_test_vectors)

In [57]:
# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_collate)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, collate_fn=pad_collate)

### LSTM

In [58]:
# hyperparameters 1 feature
# input_size = 1
# sequence_length = 0
# output_size = 3

# hyperparameters all features
input_size = len(X_features)
output_size = len(y_class_encodings)

In [59]:
# model training hyperparameters
hidden_size = 9
num_layers = 3
learning_rate = 0.001

In [60]:
model = DynamicLSTM(input_size = input_size,
                    hidden_size = hidden_size,
                    num_layers = num_layers,
                    output_size = output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Training Loop

In [62]:
# Train
n_epochs = 5
training_loss = []
training_pred = []

for epoch in range(n_epochs):
    for i, (x, y, x_lens, y_lens) in enumerate(train_dataloader):
        # forward
        out, __ = model(x, x_lens)
        print(out)
        print(y)
        # loss = criterion(out, y)
        # training_loss += [loss]
        
        # # back
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

    if (i+1) % 100 != 0:
        print(f"Epoch: {epoch+1}/{n_epochs}, Step {i+1}, Training Loss {loss.item():.4f}")

tensor([[0.3956, 0.2618, 0.3427],
        [0.3905, 0.2572, 0.3523],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427]], grad_fn=<SoftmaxBackward0>)
tensor([[1.],
        [2.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.],
        [2.],
        [1.],
        [1.],
        [2.],
        [1.]])
tensor([[0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427],
        [0.3956, 0.2618, 0.3427]

NameError: name 'loss' is not defined

In [ ]:
# Test - we don't compute gradients
with torch.no_grad():
    n_samples = 0
    n_correct = 0
    for i, (signals, labels) in enumerate(test_loader):

        sequence = signals.reshape(-1, sequence_length, input_size)
        labels = labels
        
        out = model(sequence)
        pred_val, pred_idx = torch.max(out.data, 1)
        n_samples += labels.size(0)
        n_correct += (pred_idx == labels).sum().item()
        
    print("accuracy: ", n_correct / n_samples)

y_pred = model(X_train)
train_loss = loss_fn(y_pred, y_train)
y_pred = model(X_test)
test_loss = (loss_fn(y_pred, y_test))

In [ ]:
category_idx = torch.argmax(output).item()
    return all_categories[category_idx]

print(category_from_output(output))

# Evaluation